## Importing the data 

In [2]:
import pandas as pd

/Users/samtims/.pyenv/versions/3.8.12/envs/frieda/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [9]:
users = pd.read_csv("/Users/samtims/code/Le-Wagon-Career-Week-Challenge/notebooks/follows.csv")

In [10]:
users

,user_id,follows,date
0,1,2,1993-09-01
1,2,1,1989-01-01
2,3,1,1993-07-01
3,2,3,1994-10-10
4,3,2,1995-03-01
5,4,2,1988-08-08
6,4,1,1988-08-08
7,1,4,1994-04-02
8,1,5,2000-01-01
9,5,1,2000-01-02


In [11]:
follows = pd.read_csv("/Users/samtims/code/Le-Wagon-Career-Week-Challenge/notebooks/users.csv")

In [12]:
follows.head()

,user_id,first_name,last_name,house
0,1,Harry,Potter,Gryffindor
1,2,Ron,Wesley,Gryffindor
2,3,Hermonie,Granger,Gryffindor
3,4,Ginny,Weasley,Gryffindor
4,5,Draco,Malfoy,Slytherin


### How many users are there in each house?

In [14]:
import sqlite3
conn = sqlite3.connect('/Users/samtims/code/Le-Wagon-Career-Week-Challenge/notebooks/SQL_challenge.db')
c = conn.cursor()



In [25]:
c.execute("SELECT COUNT(users.user_id), users.house FROM users GROUP BY users.house") 
question_1 = c.fetchall()
df_1 = pd.DataFrame(question_1,columns=['#_Of_Users', 'House'])
df_1.set_index('House')

,#_Of_Users
House,
Gryffindor,4
Hufflepuff,1
Ravenclaw,2
Slytherin,2


### List all following links that were created before September 1st, 1993

In [28]:
c.execute("SELECT * FROM follows WHERE date < '1993-09-01'") 
question_2 = c.fetchall()
df_2 = pd.DataFrame(question_2, columns=['user_id', 'follows', 'date'])
df_2.set_index('user_id')

,follows,date
user_id,,
2,1,1989-01-01
3,1,1993-07-01
4,2,1988-08-08
4,1,1988-08-08
5,6,1986-01-10
7,1,1990-02-02


### List all the following links established before September 1st 1993, but this time use the users first names

In [30]:
c.execute(
    """SELECT users.first_name, follows.follows, follows.date
FROM users
JOIN follows
ON users.user_id = follows.user_id
WHERE follows.date < '1993-09-01';""") 
question_3 = c.fetchall()
df_3 = pd.DataFrame(question_3, columns=['first_name', 'follows', 'date'])
df_3.set_index('first_name')

,follows,date
first_name,,
Ron,1,1989-01-01
Hermonie,1,1993-07-01
Ginny,2,1988-08-08
Ginny,1,1988-08-08
Draco,6,1986-01-10
Luna,1,1990-02-02


### Give a count of how many people followed each user as of 1999-12-31. Give the result in term of users full name, number of followers

In [36]:
c.execute(
    """SELECT users.first_name || ' ' || users.last_name AS full_name, COUNT(follows.follows) AS num_of_follows
FROM users
JOIN follows
ON users.user_id = follows.user_id
GROUP BY full_name
HAVINg follows.date < '1999-12-31'
ORDER BY num_of_follows DESC""") 
question_4 = c.fetchall()
df_4 = pd.DataFrame(question_4, columns=["name",'#_of_follows'])
df_4.set_index('name')

,#_of_follows
name,
Harry Potter,5
Hermonie Granger,3
Ginny Weasley,3
Ron Wesley,2
Luna Lovegood,2
Cho Chang,2
Cedric Diggory,1


### List the number of users each user follows

### List all rows from follows where someone from one house follows someone from a different house. Please give the result with user names.

In [41]:
c.execute(
    """select users.first_name, count(follows.follows) from users 
natural join follows 
JOIN users u2 ON u2.user_id = follows.follows 
where users.house != u2.house 
group by users.first_name;""") 
question_6 = c.fetchall()
df_6 = pd.DataFrame(question_6, columns=['name','follows'])
df_6.set_index('name')

,follows
name,
Cedric,1
Cho,2
Draco,1
Ginny,1
Harry,3
Hermonie,1
Luna,1


### List all unrequited followings (i.e. where A follows B but B does not follow A)